In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 导入工具包及设置路径

In [ ]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
from multiprocessing import Pool   

warnings.filterwarnings('ignore')
fea_pkl_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/pkl/fea_pkl/'
model_pkl_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/pkl/model_pkl/'
datasets_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/datasets/'
fea_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/Features/'
pred_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/Predict/'
sub_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/Predict/sub/'

BASE = fea_pkl_path + 'grid_part_1.pkl'
PRICE = fea_pkl_path + 'grid_part_2.pkl'
CALENDAR = fea_pkl_path + 'grid_part_3.pkl'
LAGS = fea_pkl_path + 'grid_part_4.pkl'
MEAN_ENC = fea_pkl_path + 'grid_part_5.pkl'

## 基本函数

In [ ]:
## 设定随机种子
def seed_everything(seed=0):
  random.seed(seed)
  np.random.seed(seed)

## 按照商店读取数据
def get_data_by_state(state):
  df = pd.concat([pd.read_pickle(BASE), # 9
            pd.read_pickle(PRICE).iloc[:,2:], # 11
            pd.read_pickle(CALENDAR).iloc[:,2:], # 15 
            pd.read_pickle(LAGS).iloc[:,3:], # 37
            pd.read_pickle(MEAN_ENC)[mean_features]], # 14(按商店只有6)  
            axis=1) 
  # df：(47735397, 86)由于行数相同，可以横向拼接
  df = df[df['state_id'] == state] 
  features = [col for col in list(df) if col not in remove_features] # 86 - 6(BASE) - 1(Holiday) = 79
  df = df[['id', 'd', TARGET] + features] # 79 + 3 = 82 相当于去掉了 'state_id', 'store_id', 'release', 'Holiday'(在CALENDAR里面)
  # 选训练开始的天数
  df = df[df['d'] >= START_TRAIN].reset_index(drop=True)
  return df, features


## 读取测试数据
def get_base_test():
  base_test = pd.DataFrame()

  if USE_AUX:
    model_path = model_pkl_path
  else:
    model_path=''
    
  for state_id in STATE_IDS: 
    temp_df = pd.read_pickle(model_path + 'cb_test_{}.pkl'.format(state_id)) # 不含lag + rolling特征
    temp_df['state_id'] = state_id
    base_test = pd.concat([base_test, temp_df]).reset_index(drop=True) # 纵向拼接
  
  return base_test
## 制作lag特征
def make_lag(LAG_DAY):
  lag_df = base_test[['id','d',TARGET]]
  col_name = 'sales_lag_' + str(LAG_DAY)
  lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
  return lag_df[[col_name]]
## 递归特征
def make_lag_roll(LAG_DAY):
  shift_day = LAG_DAY[0]
  roll_wind = LAG_DAY[1]
  lag_df = base_test[['id','d',TARGET]]
  col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
  lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
  return lag_df[[col_name]]
## 多线程执行，用于测试集融合
def df_parallelize_run(func, t_split):
  num_cores = np.min([N_CORES,len(t_split)])
  pool = Pool(num_cores)
  df = pd.concat(pool.map(func, t_split), axis=1)
  pool.close()
  pool.join()
  return df

# lag + rolling
SHIFT_DAY = 28
N_LAGS = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY, SHIFT_DAY + N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
  for j in [7,14,30,60]:
    ROLS_SPLIT.append([i,j])
   
USE_AUX = False
if USE_AUX:
  lgb_params['n_estimators'] = 2

VER = 1 # 设置模型的版本
SEED = 42 
seed_everything(SEED) # 消除随机性
N_CORES = psutil.cpu_count() # 可使用的CPU内核

TARGET = 'sales' # Label
START_TRAIN = 0 # 能跳过一些行(Nans/faster training)
P_HORIZON = 28 # 预测范围
USE_AUX = True # 使用预训练好的模型             

remove_features = ['id', 'state_id', 'store_id', 'release', 'd', 'Holiday', TARGET]                    
mean_features   = [ 'enc_cat_id_mean', 'enc_cat_id_std', 
             'enc_store_id_mean', 'enc_store_id_std', 
             'enc_dept_id_mean', 'enc_dept_id_std', 
             'enc_item_id_mean', 'enc_item_id_std', 
             'enc_item_id_store_id_mean', 'enc_item_id_store_id_std',
             'enc_store_id_cat_id_std', 'enc_state_id_cat_id_std',
             'enc_store_id_dept_id_mean', 'enc_store_id_dept_id_std' ]  # 14
# 按州分别训练，每个州可以按商店、类别、部门、商品的销量聚合取mean\std，可以选这14个特征

## cb模型参数

In [ ]:
!pip install catboost
from catboost import Pool, CatBoostRegressor 
# Pool是catboost中的用于组织数据的一种形式，也可以用numpy、array和dataframe，但更推荐Pool，其内存和速度都更优。
cb_params_common = { 
         'iterations' : 1524,
         'learning_rate' : 0.08868565834866991,
         'l2_leaf_reg' : 9.398124479877186,    
         'subsample' : 0.786596695482429,
         'max_depth' : 7,
         'boosting_type' : 'Plain',
         'one_hot_max_size' : 2,
         'max_bin' : 142,
         'objective' : 'RMSE',
         'eval_metric' : 'RMSE',
         'random_seed' : 42,
         'leaf_estimation_method' : 'Gradient',
         'nan_mode' : 'Max',
         'min_data_in_leaf' : 11965,
         'boost_from_average' : False,
         'verbose' : 100,
         'early_stopping_rounds' : 50 # 用earlystopping后训练时间更短，可以有效避免过拟合，得到的模型准确率更高。
        }

     |████████████████████████████████| 64.8MB 69kB/s 


## 每个州单独训练(按州TX聚合效果差，只聚合CA和WI)

In [ ]:
STATE_IDS = ['CA','WI'] 
END_TRAIN = 1941

In [ ]:
# features_columns = ['item_id', 'dept_id', 'cat_id', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_60', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_store_id_mean', 'enc_store_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'enc_item_id_store_id_mean', 'enc_item_id_store_id_std', 'enc_store_id_cat_id_std', 'enc_state_id_cat_id_std', 'enc_store_id_dept_id_mean', 'enc_store_id_dept_id_std']
# 79

In [ ]:
for state_id in STATE_IDS:
  print('Train', state_id)
  grid_df, features_columns = get_data_by_state(state_id) 
  ########################## 添加节假日特性 #############################
  calendar_win = pd.read_csv(datasets_path + 'calendar.csv')[['event_name_2','event_name_1','d']]
  # 添加特征
  event = calendar_win[['event_name_2','event_name_1']].fillna('') 
  # 整体上移一天
  event_before1 = pd.DataFrame()
  event_up_one = event.shift(-1).fillna('') 
  event_before1['event_name_2'] = event_up_one['event_name_2'].apply(lambda x: x + '_before1' if x != '' else x)
  event_before1['event_name_1'] = event_up_one['event_name_1'].apply(lambda x: x + '_before1' if x != '' else x)
  # 整体上移两天
  event_before2 = pd.DataFrame()
  event_up_two = event.shift(-2).fillna('')   
  event_before2['event_name_2'] = event_up_two['event_name_2'].apply(lambda x: x + '_before2' if x != '' else x)
  event_before2['event_name_1'] = event_up_two['event_name_1'].apply(lambda x: x + '_before2' if x != '' else x)
  event_win = event_before1 + event_before2

  calendar_win.drop(columns=['event_name_1','event_name_2'],inplace=True)

  # 转换数据类型
  calendar_win['event_name_1_win'] = event_win['event_name_1'].astype('category')
  calendar_win['event_name_2_win'] = event_win['event_name_2'].astype('category')
  calendar_win['d'] = calendar_win['d'].apply(lambda x: x.split('_',2)[1]).astype(np.int16)

  # 小融入大 merge
  grid_df = grid_df.merge(calendar_win, on='d', how='left')
  features_columns = features_columns + ['event_name_1_win','event_name_2_win']

  if state_id == 'CA':
    cb_params = cb_params_common
  elif state_id == 'WI':
    cb_params = cb_params_common

  train_mask = grid_df['d'] <= END_TRAIN - P_HORIZON # 1<= train <=1941-28 
  valid_mask = (grid_df['d'] > END_TRAIN - P_HORIZON) & (grid_df['d'] <= END_TRAIN) # 1941-28< valid <=1941
  preds_mask = grid_df['d'] > END_TRAIN - 100 # 1941-100< test <=1969 

  categorical_features_indices = ['item_id','dept_id','cat_id','snap_CA','snap_TX','snap_WI',
                    'event_name_1','event_type_1','event_name_2','event_type_2',
                    'event_name_1_win','event_name_2_win']
  for i in categorical_features_indices:
    grid_df[i] = grid_df[i].astype('object') # 类别数据的类型要是object
    grid_df[i] = grid_df[i].fillna('no') # 注意：cb中不能出现空值，类别列也不可以

  train_pool = Pool(grid_df[train_mask][features_columns], grid_df[train_mask][TARGET], cat_features = categorical_features_indices) 
  valid_pool = Pool(grid_df[valid_mask][features_columns], grid_df[valid_mask][TARGET], cat_features = categorical_features_indices)

  grid_df = grid_df[preds_mask].reset_index(drop=True) 
  keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
  # 细节：在测试集中去掉了递归特征
  grid_df = grid_df[keep_cols] 
  grid_df.to_pickle(model_pkl_path + 'cb_test_{}.pkl'.format(state_id))
  
  # 训练模型
  seed_everything(SEED)
  cb_model = CatBoostRegressor(**cb_params) 
  cb_model.fit(train_pool, eval_set = valid_pool) 
  # 存储模型为二进制文件，下次读取的时候，速度更快
  model_name = model_pkl_path + 'cb_model_{}_v{}.bin'.format(state_id, str(VER))
  pickle.dump(cb_model, open(model_name, 'wb'))

  del grid_df, train_pool, valid_pool, cb_model

  MODEL_FEATURES = features_columns

Train CA
0:	learn: 4.5785288	test: 3.6324570	best: 3.6324570 (0)	total: 5.25s	remaining: 2h 13m 18s
100:	learn: 2.4862629	test: 1.9786582	best: 1.9786582 (100)	total: 6m 12s	remaining: 1h 27m 34s
200:	learn: 2.4363563	test: 1.9637915	best: 1.9637915 (200)	total: 12m 25s	remaining: 1h 21m 49s
300:	learn: 2.4115216	test: 1.9573065	best: 1.9572969 (299)	total: 18m 42s	remaining: 1h 16m 2s


## 预测

In [ ]:
# 训练特征 79 + 2 = 81 
# MODEL_FEATURES = ['item_id', 'dept_id', 'cat_id', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_60', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_store_id_mean', 'enc_store_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'enc_item_id_store_id_mean', 'enc_item_id_store_id_std', 'enc_store_id_cat_id_std', 'enc_state_id_cat_id_std', 'enc_store_id_dept_id_mean', 'enc_store_id_dept_id_std', 'event_name_1_win', 'event_name_2_win']

In [ ]:
USE_AUX = True
END_TRAIN = 1941
all_preds = pd.DataFrame()
base_test = get_base_test() # [2731904 rows x 73 columns]
main_time = time.time()
       
for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # 加上测试集"lag+rolling"特征(12个)
    grid_df = base_test.copy() 
    grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)

    for state_id in STATE_IDS:
      model_bin = 'cb_model_{}_v{}.bin'.format(state_id, str(VER))
      if USE_AUX:
        model_path = model_pkl_path + model_bin
      # 加载训练好的模型
      cb_model = pickle.load(open(model_path, 'rb'))
      
      day_mask = base_test['d'] == END_TRAIN + PREDICT_DAY 
      store_mask = base_test['state_id'] == state_id 
      mask = (day_mask) & (store_mask) 

      categorical_features_indices = ['item_id','dept_id','cat_id','snap_CA','snap_TX','snap_WI',
                        'event_name_1','event_type_1','event_name_2','event_type_2',
                        'event_name_1_win','event_name_2_win']
      for i in categorical_features_indices:
        grid_df[i] = grid_df[i].astype('object')
        grid_df[i] = grid_df[i].fillna('no') 

      base_test[TARGET][mask] = cb_model.predict(grid_df[mask][MODEL_FEATURES])
      # 比如预测得到的1942这一天的销量，把它填充到base_test的"sales"对应1942这一天"NAN"值
      # 所以，base_test的"sales"一直在变，所以下面预测1943这一天的销量，会用到前面1942这一天的预测值

    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id', 'F{}'.format(str(PREDICT_DAY))]
    if 'id' in list(all_preds):
      all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
      all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
            ' %0.2f min total |' % ((time.time() - main_time) / 60),
            ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds.to_csv(pred_path + 'per_state(WI+CA)_pred(cb).csv')

# 用于生成提交预测的文件
submission = pd.read_csv(datasets_path + 'sample_submission.csv')[['id']] # [[]] 返回DataFrame [] 返回Series
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv(datasets_path + 'submission_by_states(cb).csv'), index=False) 

## 某个州进行贝叶斯调参

In [ ]:
STATE_IDS = 'TX'
grid_df, features_columns = get_data_by_state(STATE_IDS)
day = 1941
grid_df1 = grid_df[grid_df['d'] <= day] 
del grid_df

!pip install bayesian-optimization

import lightgbm as lgb
from bayes_opt import BayesianOptimization
# 定义RMSE
def rmse(y, y_pred):
  return np.sqrt(np.mean(np.square(y - y_pred)))

df = grid_df1
fe = features_columns
# 1000到1941-28-28 作为训练集
tr_x, tr_y = df[(df['d'] >= 1000) & (df['d'] <= (day-28-28))][fe], df[(df['d'] >= 1000) & (df['d'] <= (day-28-28))]['sales'] 
# 1941-28-28到1941-28 作为验证集
vl_x, vl_y = df[(df['d'] > (day-28-28)) & (df['d'] <= (day-28))][fe], df[(df['d'] > (day-28-28)) & (df['d'] <= (day-28))]['sales']

train_data = lgb.Dataset(tr_x, label = tr_y)
valid_data = lgb.Dataset(vl_x, label = vl_y)

# 1914到1941 作为测试集，用训练的模型预测它的"sales"，再与真实的"sales"计算RMSE
test_df = df[df['d'] > day-28].reset_index(drop=True) 

# 定义黑盒函数
def lgb_cv(tweedie_variance_power, subsample, subsample_freq, learning_rate, num_leaves, min_data_in_leaf,
      feature_fraction, max_bin, n_estimators, lambda_l2, sub_row, sub_feature, bagging_freq):


    lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': tweedie_variance_power,
                    'metric': 'rmse',
                    'subsample': subsample,
                    'subsample_freq': int(subsample_freq),
                    'learning_rate': learning_rate,
                    'num_leaves': int(num_leaves),
                    'min_data_in_leaf': int(min_data_in_leaf),
                    'feature_fraction': feature_fraction,
                    'max_bin': int(max_bin),
                    'n_estimators': int(n_estimators),
                    'boost_from_average': False,
                    'seed': 42,  
                    'lambda_l2':lambda_l2,
                    'sub_row':sub_row,  
                    'sub_feature':sub_feature,
                    'bagging_freq':int(bagging_freq)          
                }
    stimator = lgb.train(lgb_params, train_data, num_boost_round = 10000, valid_sets = [valid_data], verbose_eval = 100, early_stopping_rounds = 100) 
    # valid_sets = [train_data, valid_data] 这个是用来显示training's rmse和valid_1's rmse，这里不用显示，就不要了
    # verbose_eval : 迭代多少次打印
    test_df['preds'] = stimator.predict(test_df[fe], num_iteration = stimator.best_iteration)
    base_score = rmse(test_df['sales'], test_df['preds'])

    return -base_score

# 给定超参数搜索空间
opt = BayesianOptimization(
                lgb_cv,
                {
                    'tweedie_variance_power': (1, 1.3),
                    'subsample': (0.5, 1.0),
                    'subsample_freq': (0, 5),
                    'learning_rate': (0, 1),
                    'num_leaves':(2**10-1,2**15-1),
                    'min_data_in_leaf':(2**10-1,2**15-1),
                    'feature_fraction':(0.4,1),
                    'max_bin':(80,150),
                    'n_estimators':(1000,1700),
                    'lambda_l2':(0,0.2),
                    'sub_row':(0.6,1),
                    'sub_feature':(0.6,1.0),
                    'bagging_freq':(0,5),
                }
              )
opt.maximize(n_iter = 20) # 最大化黑盒函数
rf_bo.max # 返回黑盒函数值最大的超参数